# Open project: data cleanup

### Isaac Newell

This notebook gets and cleans data from:
* _League of Conservation Voters_, on the voting records of congressmen
* _OpenSecrets.org_, on the industry campaign donations to congressmen

In conjunction with data from Yale's Program on Climate Change Communication, these data will be used to assess the primary question: *How does industry money from Oil, Gas, and Coal effect the voting of the receiving members of congress? How does it pull them away from the opinion of their constituents?*

For a much more extensive explanation, see my webpage itself.

In [2]:
import requests
import json
from bs4 import BeautifulSoup
import csv

First we'll start with the LCV data. The data is contained inside a table-like structure, with a weird way of toggling between pages of data and filtering out other data. So, using BeautifulSoup, we can get all the divs that may contain useful information, and check that the data corresponds to a member of the House by ensuring that there is a specific district attached.

In [3]:
r = requests.get("http://scorecard.lcv.org/members-of-congress")

html_doc = r.text

soup = BeautifulSoup(html_doc, 'html.parser')

container = soup.find("div",attrs={"id":"moc-list-table-data"})

divs = container.find_all("div")

# data will be saved into scorecard_data
scorecard_data = []
for div in divs:
    # get the state+district, i.e. MA-04
    st = div.find("span",class_="mocState").get_text()
    # check if st is an at large district
    if st[-2:].isdigit() or (st[-2:] == "AL" and st != "AL"):
        name = div.find("span",class_="mocName").get_text()
        # These congressmen either came into our out of office in 2016.
        # The dataset had two people in their districts, and I only wanted one each.
        # Thus I picked the ones with the most extensive voting record.
        if (name in ["Takai, Mark", "Comer, James", "Fattah, Chaka"]):
            continue
        row = []
        for c in ["mocName","mocParty","mocState","mocRating","mocLifetimeRating"]:
            row.append(div.find("span",class_=c).get_text())
        row.append(div.find("span",class_="mocState").get("sort"))
        scorecard_data.append(row)
        
# Sort data
from operator import itemgetter
scorecard_data.sort(key=itemgetter(-1))

# Write data to csv
col_headers = ["rep", "party", "district", "score_2016", "score_lifetime"]
with open('climate_scorecard.csv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(col_headers)
    for row in scorecard_data:
        writer.writerow(row[:-1])

Now, onwards to the OpenSecrets data. For this, by inspecting the webpage I gleaned that I needed to do a little string processing. One of the fields on the webpage contains data in the form of LASTNAME, FIRSTNAME, (PARTY-STATE), i.e. McCarthy, Kevin (R-CA). Opensecrets has a sort feature, but it only sorts by state, not by district. Thus we will have to process this string and use our previous data (in scorecard_data) to determine the corresponding district.

In [4]:
def get_district(rep):
    name = rep[0:rep.find("(")-1]
    for row in scorecard_data:
        n = row[0]
        if n[-1] == ".":
            n = row[0][:-1]
        if n == name:
            return row[2][-2:]
    return "Missing"
def get_state(rep):
    state = rep[rep.find("-",rep.find("("))+1:rep.find(")")]
    return state

Test it out to make sure it works...

In [5]:
print(get_district("McCarthy, Kevin (R-CA)"))
print(get_state("McCarthy, Kevin (R-CA)"))

print(get_district("Kelly, Trent (R-MS)"))
print(get_state("Kelly, Trent (R-MS)"))

23
CA
01
MS


Looks correct!

Now, our approach will be to iterate through the congressmen, sorted by state, and place them into the proper index of a 435-dimensional vector. That requires us to know the alphabetical order of the states as well as the number of districts in each state:

In [22]:
dists = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY",
        "LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND",
        "OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]


num_dists = {"AL": 7,
             "AK": 1,
             "AZ": 9,
             "AR": 4,
             "CA": 53,
             "CO": 7,
             "CT": 5,
             "DE": 1,
             "FL": 27,
             "GA": 14,
             "HI": 2,
             "ID": 2,
             "IL": 18,
             "IN": 9,
             "IA": 4,
             "KS": 4,
             "KY": 6,
             "LA": 6,
             "ME": 2,
             "MD": 8,
             "MA": 9,
             "MI": 14,
             "MN": 8,
             "MS": 4,
             "MO": 8,
             "MT": 1,
             "NE": 3,
             "NV": 4,
             "NH": 2,
             "NJ": 12,
             "NM": 3,
             "NY": 27,
             "NC": 13,
             "ND": 1,
             "OH": 16,
             "OK": 5,
             "OR": 5,
             "PA": 18,
             "RI": 2,
             "SC": 7,
             "SD": 1,
             "TN": 9,
             "TX": 36,
             "UT": 4,
             "VT": 1,
             "VA": 11,
             "WA": 10,
             "WV": 3,
             "WI": 8,
             "WY": 1
            }

Do a quick check to make sure the districts total to 435:

In [23]:
total = 0
for k in num_dists.keys():
    total += num_dists[k]
print(total)

435


A bit more preliminary work, we'll have to convert from dollars to integer values. The to_int method does that.

In [8]:
def to_int(dollars):
    s = ""
    for i in range(len(dollars)):
        if dollars[i].isdigit():
            s += dollars[i]
    return int(s)

In [9]:
print(to_int("$4,356,098"))

4356098


Now, we are ready to actually parse the data. The following function will take the url (seperate for each industry, Oil, Gas, and Coal), and the number of pages, since the data is seperated alphabetically by some number of pages.

In [24]:
def get_opensecrets_data(url,numpages):
    data_col = [0]*435
    curr_state = "AL"
    curr_state_num = 0
    state_start_idx = 0
    for p in range(1,numpages+1):
        # make a request for the current page
        r = requests.get(url+"&page="+str(p))
        doc = r.text
        soup = BeautifulSoup(doc,"html.parser")
        tab = soup.find("table",class_="datadisplay")
        # get all the trs, containing the data of use
        trs = tab.find_all("tr")[1:]
        for tr in trs:
            # So, some districts may be missing, districts within a state may not be in order
            # Thus, we must keep track of what state we are on.
            tds = tr.find_all("td")
            state = get_state(tds[0].get_text())
            if state not in num_dists.keys():
                continue
            # If state changes, update the start of the state range.
            # Note that some states might not be included at all so you may have to
            # add the number of districts in several alphabetically consecutive states
            if state != curr_state:
                new_state_num = dists.index(state)
                for n in range(curr_state_num,new_state_num):
                    state_start_idx += num_dists[dists[n]]
                curr_state_num = new_state_num
                curr_state = state
                print("starting " + str(state) + " at: " + str(state_start_idx))
            dist = get_district(tds[0].get_text())
            money = tds[1].get_text()
            # address at-large districts
            if dist == "AL":
                dist = 1
            if dist != "Missing":
                dist = int(dist)
                data_col[state_start_idx+dist-1] = to_int(money)
    return data_col

Just for debugging purposes, run the following below to make sure that every industry it is called on, get_opensecrets_data starts the index for each state at the same place; the order of states and number of districts is constant.

In [26]:
print(get_opensecrets_data("https://www.opensecrets.org/industries/summary.php?ind=E01&cycle=2016&recipdetail=H&sortorder=S&mem=Y",4))

starting AK at: 7
starting AZ at: 8
starting AR at: 17
starting CA at: 21
starting CO at: 74
starting CT at: 81
starting DE at: 86
starting FL at: 87
starting GA at: 114
starting HI at: 128
starting ID at: 130
starting IL at: 132
starting IN at: 150
starting IA at: 159
starting KS at: 163
starting KY at: 167
starting LA at: 173
starting ME at: 179
starting MD at: 181
starting MA at: 189
starting MI at: 198
starting MN at: 212
starting MS at: 220
starting MO at: 224
starting MT at: 232
starting NE at: 233
starting NV at: 236
starting NH at: 240
starting NJ at: 242
starting NM at: 254
starting NY at: 257
starting NC at: 284
starting ND at: 297
starting OH at: 298
starting OK at: 314
starting OR at: 319
starting PA at: 324
starting RI at: 342
starting SC at: 344
starting SD at: 351
starting TN at: 352
starting TX at: 361
starting UT at: 397
starting VT at: 401
starting VA at: 402
starting WA at: 413
starting WV at: 423
starting WI at: 426
starting WY at: 434
[33450, 21950, 12500, 18200, 0

Now write the data to a csv file. We will call get_opensecrets_data on the appropriate urls for Oil&Gas, natural gas pipelines, and coal and the appropriate number of pages (gleaned from visiting each link).

In [25]:
col_headers = ["rep","party","district","oil_gas","pipelines","coal"]
           
oil_gas_col = get_opensecrets_data("https://www.opensecrets.org/industries/summary.php?ind=E01&cycle=2016&recipdetail=H&sortorder=S&mem=Y",4)
pipelines_col = get_opensecrets_data("https://www.opensecrets.org/industries/summary.php?ind=E1140&cycle=2016&recipdetail=H&sortorder=S&mem=Y",3)
coal_col = get_opensecrets_data("https://www.opensecrets.org/industries/summary.php?ind=E1210&cycle=2016&recipdetail=H&sortorder=S&mem=Y",1)

with open('lobbying_2016.csv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(col_headers)
    for i,row in enumerate(scorecard_data):
        writer.writerow(row[:3]+[oil_gas_col[i]]+[pipelines_col[i]]+[coal_col[i]])
    

starting AK at: 7
starting AZ at: 8
starting AR at: 17
starting CA at: 21
starting CO at: 74
starting CT at: 81
starting DE at: 86
starting FL at: 87
starting GA at: 114
starting HI at: 128
starting ID at: 130
starting IL at: 132
starting IN at: 150
starting IA at: 159
starting KS at: 163
starting KY at: 167
starting LA at: 173
starting ME at: 179
starting MD at: 181
starting MA at: 189
starting MI at: 198
starting MN at: 212
starting MS at: 220
starting MO at: 224
starting MT at: 232
starting NE at: 233
starting NV at: 236
starting NH at: 240
starting NJ at: 242
starting NM at: 254
starting NY at: 257
starting NC at: 284
starting ND at: 297
starting OH at: 298
starting OK at: 314
starting OR at: 319
starting PA at: 324
starting RI at: 342
starting SC at: 344
starting SD at: 351
starting TN at: 352
starting TX at: 361
starting UT at: 397
starting VT at: 401
starting VA at: 402
starting WA at: 413
starting WV at: 423
starting WI at: 426
starting WY at: 434
starting AZ at: 8
starting AR 